# Training CIAP Contrastive Model on Google Colab
This notebook demonstrates how to train the CIAP contrastive model on Google Colab. The trained weights for the ImageEncoder and AudioEncoder models will be saved for later use.

## Section 1: Install Required Libraries
Install necessary libraries such as PyTorch, PyYAML, and tqdm using pip.

In [ ]:
# Install required libraries
!pip install torch torchvision torchaudio
!pip install pyyaml tqdm

## Section 2: Clone the Repository and Set Up Environment
Clone the AudioLDM repository, navigate to the required directory, and install dependencies.

In [ ]:
# Clone the AudioLDM repository
!git clone https://github.com/haoheliu/AudioLDM.git
%cd AudioLDM

# Install dependencies
!pip install -r requirements.txt

## Section 3: Load Configuration
Load the configuration file (ciap_config.yaml) using PyYAML and parse the training parameters.

In [ ]:
import yaml

def load_config(config_path):
    with open(config_path, "r") as f:
        return yaml.safe_load(f)

# Load the configuration file
config_path = "audioldm/ciap/configs/ciap_config.yaml"
config = load_config(config_path)

# Display the configuration
print(config)

## Section 4: Define Dataset and DataLoader
Use the `PairedImageAudioDataset` class to define the dataset and create a DataLoader for training.

In [ ]:
from torch.utils.data import DataLoader
from audioldm.ciap.datasets.paired_image_audio_dataset import PairedImageAudioDataset

# Extract dataset parameters from the configuration
dataset_cfg = config.get("dataset", {})
dataset_path = dataset_cfg.get("path", "")
image_ext = dataset_cfg.get("image_extension", ".png")
audio_ext = dataset_cfg.get("audio_extension", ".wav")

# Define the dataset and DataLoader
dataset = PairedImageAudioDataset(dataset_path, image_ext=image_ext, audio_ext=audio_ext)
batch_size = config["training"].get("batch_size", 32)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

## Section 5: Initialize Models
Initialize the `ImageEncoder` and `AudioEncoder` models using the configuration parameters.

In [ ]:
import torch
from audioldm.ciap.models.image_encoder import ImageEncoder
from audioldm.ciap.models.audio_encoder import AudioEncoder

# Initialize models
device = torch.device(config.get("device", "cpu"))
image_encoder = ImageEncoder(config["model"]["image_encoder"]).to(device)
audio_encoder = AudioEncoder(config["model"]["audio_encoder"]).to(device)

# Set models to training mode
image_encoder.train()
audio_encoder.train()

## Section 6: Define Loss Function and Optimizer
Set up the `ContrastiveLoss` function and Adam optimizer for training.

In [ ]:
import torch.optim as optim
from audioldm.ciap.losses.contrastive_loss import ContrastiveLoss

# Define loss function and optimizer
criterion = ContrastiveLoss()
lr = config["training"].get("learning_rate", 1e-3)
optimizer = optim.Adam(list(image_encoder.parameters()) + list(audio_encoder.parameters()), lr=lr)

## Section 7: Training Loop
Implement the training loop with tqdm for progress tracking, and calculate the loss for each batch.

In [ ]:
from tqdm import tqdm

# Training parameters
num_epochs = config["training"].get("num_epochs", 10)

# Training loop
for epoch in range(1, num_epochs + 1):
    running_loss = 0.0
    loop = tqdm(enumerate(dataloader, start=1), total=len(dataloader),
                desc=f"Epoch {epoch}/{num_epochs}", leave=True)
    for batch_idx, (images, audios) in loop:
        images = images.to(device)
        audios = audios.to(device)

        optimizer.zero_grad()

        # Use encode() if available; fallback to forward()
        img_emb = image_encoder.encode(images) if hasattr(image_encoder, "encode") else image_encoder(images)
        aud_emb = audio_encoder.encode(audios) if hasattr(audio_encoder, "encode") else audio_encoder(audios)

        # Labels: i-th image matches i-th audio
        labels = torch.arange(img_emb.size(0), device=device)

        loss = criterion(img_emb, aud_emb, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        avg_loss = running_loss / batch_idx
        loop.set_postfix({"avg_loss": f"{avg_loss:.4f}", "batch_loss": f"{loss.item():.4f}"})

    print(f"Epoch {epoch} finished. EpochLoss: {running_loss/len(dataloader):.4f}")

## Section 8: Save Trained Weights
Save the trained weights of the `ImageEncoder` and `AudioEncoder` models to the Colab environment.

In [ ]:
import os

# Save trained weights
output_dir = "ciap_trained_weights"
os.makedirs(output_dir, exist_ok=True)

img_path = os.path.join(output_dir, "ciap_image_encoder.pt")
aud_path = os.path.join(output_dir, "ciap_audio_encoder.pt")
torch.save(image_encoder.state_dict(), img_path)
torch.save(audio_encoder.state_dict(), aud_path)

print(f"Saved image encoder -> {img_path}")
print(f"Saved audio encoder  -> {aud_path}")